In [1]:
%reset -f

In [2]:
import numpy as np
import pandas as pd
import re
import spacy

data_path = '/Users/ayemyatwinshwe/repos/data/trading_data/'

In [3]:
nlp = spacy.load('en_core_web_sm', disable=['ner'])
nlp.max_length = 6000000

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    """
        1. Replace url in the text with 'urlplaceholder'
        2. Tokenize the text removing stop words, digits, punctuations, space, symbols
    """
    urls = re.findall(url_regex, text)
    for url in urls:
        text = text.replace(url, "urlplaceholder")
    doc = nlp(text)
    tokens = [token.text.lower() for token in doc if not any ([token.is_stop,
                                                               token.is_digit,
                                                               not token.is_alpha,
                                                               token.is_punct, 
                                                               token.is_space,
                                                               token.lemma_ == '_PRON_',
                                                               token.pos_ in ['PUNCT', 'SYM', 'X']])]
    
    return tokens

In [5]:
df_earning_call = pd.read_parquet(data_path + 'earning_calls_1')
df_earning_call.head()

,quarter,year,date,content,prev_day,next_7day,prev_adjClose,next_adjClose
symbol,,,,,,,,
A,1,2019,2019-03-04,"Operator: Good day, ladies and gentlemen, and ...",2019-02-20,2019-03-04,77.049599,78.589417
A,2,2019,2019-05-24,"Operator: Good day, ladies and gentlemen. And ...",2019-05-14,2019-05-24,74.701042,67.182785
A,3,2019,2019-08-26,"Operator: Good afternoon, and welcome to the A...",2019-08-14,2019-08-26,64.602371,68.591385
A,4,2019,2019-12-05,"Operator: Good afternoon, and welcome to the A...",2019-11-25,2019-12-05,79.221558,80.475121
A,1,2020,2020-02-28,Operator: Good afternoon and welcome to the Ag...,2020-02-18,2020-02-28,83.869682,76.233475


In [6]:
df_transcript = df_earning_call[['content']]
df_transcript.reset_index(drop=True, inplace=True)
del df_earning_call

In [8]:
df_transcript['tokens'] = df_transcript['content'].apply(tokenize)

In [9]:
df_transcript

,content,tokens
0,"Operator: Good day, ladies and gentlemen, and ...","[operator, good, day, ladies, gentlemen, welco..."
1,"Operator: Good day, ladies and gentlemen. And ...","[operator, good, day, ladies, gentlemen, welco..."
2,"Operator: Good afternoon, and welcome to the A...","[operator, good, afternoon, welcome, agilent, ..."
3,"Operator: Good afternoon, and welcome to the A...","[operator, good, afternoon, welcome, agilent, ..."
4,Operator: Good afternoon and welcome to the Ag...,"[operator, good, afternoon, welcome, agilent, ..."
...,...,...
26232,"Operator: Good afternoon, and welcome to the Z...","[operator, good, afternoon, welcome, zynex, ye..."
26233,Operator: Good day and welcome to the Zynex Fi...,"[operator, good, day, welcome, zynex, quarter,..."
26234,"Operator: Good day, and welcome to the Zynex S...","[operator, good, day, welcome, zynex, second, ..."
26235,Operator: Good afternoon and welcome to the Zy...,"[operator, good, afternoon, welcome, zynex, fo..."


In [13]:
df_transcript[['tokens']].to_parquet(data_path + 'earning_call_tokens', engine='pyarrow', compression='gzip')